In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
from IPython import get_ipython

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.impute import SimpleImputer

In [3]:
findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = (SparkConf().set("spark.driver.maxResultSize", "4g"))

# Create new context
sc = SparkContext(conf=conf)


# sc = SparkContext("local", "Simple App")

# Create SparkSession 
spark = SparkSession.builder \
       .master("local[2]") \
       .appName("test") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

df_population = spark.sql("SELECT * FROM population")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 15:35:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/30 15:35:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/30 15:35:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
def parts_replacement_aggregation(series):
    """
    Custom aggregation function to check if any value is zero.
    If no zero is found, return 0; otherwise, return 1.
    """
    if (series == 0).any():
        return 1
    else:
        return 0

In [5]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

# Parting cleaned and normalized dataset into train and test data

In [6]:
selected_features_from_population = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s]
all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features


df = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv', sep=',', names=all_columns_names, header=None)
# Display the DataFrame
print("DataFrame loaded successfully:")
display(df)
print(f"Number of rows read: {len(df)}")



# Convert columns with '_dtc' in their names to numeric
for col in df.columns:
    if '_dtc' in col:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    if '_parts' in col:
        df[col] = pd.to_numeric(df[col], errors='coerce')
# Ensure columns used in aggregation are numeric
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Ensure columns used in aggregation are non-numeric
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns 

# print(numeric_cols)
# print(non_numeric_cols)

# Normalizing non-numeric chunk of the dataframe
selected_numeric_features_for_this_VIN = [s for s in df.columns if '_dtc' in s or '_parts' in s]
df[selected_numeric_features_for_this_VIN] = df[selected_numeric_features_for_this_VIN].apply(lambda iterator: ((iterator - iterator.mean())/(iterator.max() - iterator.min())).round(3))


# Define the feature columns and target columns
feature_columns = list(range(593, 721))  # Columns 593 to 722 as features
target_columns = [721, 722]  # Columns 721 and 722 as targets

# Split the data into features and targets
X = df.iloc[:, feature_columns]
y = df.iloc[:, target_columns]


VINs = df['VIN'].unique()

# Split the VINs into train and test sets
VINs_train, VINs_test = train_test_split(VINs, test_size=0.2, random_state=42)

# Split the original dataframe into train and test dataframes based on the VINs
train_data = df[df['VIN'].isin(VINs_train)]
test_data = df[df['VIN'].isin(VINs_test)]

# Write the test data to a CSV file
# test_data.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/test_data.csv', index=False)  # Change this to your desired file path

# Display the shapes of the train and test data
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


DataFrame loaded successfully:


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4V4NC9EH6KN904736,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102861,4V4NC9DG4GN972274,D11,405,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1102862,4V4NC9DG4GN972274,D11,405,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1102863,4V4NC9DG4GN972274,D11,405,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1102864,4V4NC9EH5KN207395,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Number of rows read: 1102866
Train data shape: (870703, 723)
Test data shape: (232163, 723)


# Aggregating Train and Test Data

In [7]:
# Create the aggregation dictionary for each condition
ENGINE_HP_agg = {col: 'first' for col in numeric_cols if 'ENGINE_HP' in col}

dtc_agg = {col: 'sum' for col in numeric_cols if '_dtc' in col}

'''
Different strategies to aggregate target: boolean or sum
'''
parts_agg = {col: parts_replacement_aggregation for col in numeric_cols if '_parts' in col}
# parts_agg = {col: 'sum' for col in numeric_cols if '_parts' in col}

# Create the aggregation dictionary for non-numeric columns
# After aggregation the existance of calendar_day values has no meaning, so the column will be dropped.
non_numeric_agg = {col: 'first' for col in non_numeric_cols if 'VIN' not in col and 'calendar_day' not in col}


# Combine the dictionaries
aggregation_dict = {**ENGINE_HP_agg, **non_numeric_agg, **dtc_agg, **parts_agg}

# Ensure the aggregation dictionary is not empty
if not aggregation_dict:
    raise ValueError("No columns found for aggregation")

# Group by 'VIN' and apply aggregation functions
aggregated_train_df = train_data.groupby('VIN').agg(aggregation_dict).reset_index()
aggregated_test_df = test_data.groupby('VIN').agg(aggregation_dict).reset_index()

# Step 1: Remove rows where VIN length is not exactly 17 characters
aggregated_train_df = aggregated_train_df[aggregated_train_df['VIN'].str.len() >= 17]
aggregated_test_df = aggregated_test_df[aggregated_test_df['VIN'].str.len() >= 17]

# Step 2: Clean VINs with trailing spaces and numbers
aggregated_train_df['VIN'] = aggregated_train_df['VIN'].str.replace(r'(\s[0-9])$', '', regex=True)
aggregated_test_df['VIN'] = aggregated_test_df['VIN'].str.replace(r'(\s[0-9])$', '', regex=True)

# Step 3: Remove any remaining rows where VIN length is not exactly 17 characters
aggregated_train_df = aggregated_train_df[aggregated_train_df['VIN'].str.len() == 17]
aggregated_test_df = aggregated_test_df[aggregated_test_df['VIN'].str.len() == 17]

aggregated_train_df = aggregated_train_df.iloc[:, [0] + [2, 1] + list(range(3, len(aggregated_train_df.columns)))]
aggregated_test_df = aggregated_test_df.iloc[:, [0] + [2, 1] + list(range(3, len(aggregated_test_df.columns)))]

aggregated_train_df.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/aggregated_train_df.csv', index = None, mode = 'w', header=False)
aggregated_test_df.to_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/aggregated_test_df.csv', index = None, mode = 'w', header=False)


print("Aggregated train and test data per VIN:")
print("Aggregated train data shape:", aggregated_train_df.shape)
print("Aggregated test data shape:", aggregated_test_df.shape)

Aggregated train and test data per VIN:
Aggregated train data shape: (518, 722)
Aggregated test data shape: (132, 722)


In [8]:
print(aggregated_train_df.iloc[:, 0:5])  # col[0] is VIN, col[1] to col[590] KOLA features. col[591] to col[720] are calculated features. 

                   VIN ENGINE_SIZE  ENGINE_HP VEH_TYPE _KOLA_01X
24   1M1AN07Y0JM027470         D13        505      CHU  WTDF24.5
25   1M1AN07Y1JM028191         D13        505      CHU  WTDF24.5
26   1M1AN07Y2HM023723         D13        500      CHU  WTDF24.5
27   1M1AN07Y3FM021301         D13        445      CHU  WTDF24.5
28   1M1AN07Y3JM027320         D13        445      CHU  WTDF24.5
..                 ...         ...        ...      ...       ...
538  4V5RC9DGXHN970351         D11        405      VAH  WTDF22.5
539  4V5RC9DH8JN995389         D11        425      VAH  WTDF22.5
540  4V5RC9EH5FN181268         D13        425      VAH  WTDF22.5
541  4V5RC9EH6GN960115         D13        455      VAH  WTDF22.5
542  4V5RC9EH6KN235039         D13        425      VAH  WTDF22.5

[518 rows x 5 columns]
